In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import datetime
from pathlib import Path

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow as tf

%matplotlib inline


Bad key "text.kerning_factor" on line 4 in
C:\Users\Andy L\anaconda3\envs\dev\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution
C:\Users\Andy L\anaconda3\envs\dev\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Andy L\anaconda3\envs\dev\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Andy L\anaconda3\envs\dev\lib\site-packages\

In [12]:
# Import Dataset
file_path_21 = Path("Resources/batting_stats_2021.csv")
file_path_20 = Path("Resources/batting_stats_2020.csv")
file_path_19 = Path("Resources/batting_stats_2019.csv")
file_path_18 = Path("Resources/batting_stats_2018.csv")
file_path_17 = Path("Resources/batting_stats_2017.csv")
file_path_16 = Path("Resources/batting_stats_2016.csv")
file_path_15 = Path("Resources/batting_stats_2015.csv")
file_path_14 = Path("Resources/batting_stats_2014.csv")
file_path_13 = Path("Resources/batting_stats_2013.csv")
file_path_12 = Path("Resources/batting_stats_2012.csv")
file_path_11 = Path("Resources/batting_stats_2011.csv")
file_path_10 = Path("Resources/batting_stats_2010.csv")

# Read into Dataset
stats_21_df = pd.read_csv(file_path_21, index_col = 0)
stats_20_df = pd.read_csv(file_path_20, index_col = 0)
stats_19_df = pd.read_csv(file_path_19, index_col = 0)
stats_18_df = pd.read_csv(file_path_18, index_col = 0)
stats_17_df = pd.read_csv(file_path_17, index_col = 0)
stats_16_df = pd.read_csv(file_path_16, index_col = 0)
stats_15_df = pd.read_csv(file_path_15, index_col = 0)
stats_14_df = pd.read_csv(file_path_14, index_col = 0)
stats_13_df = pd.read_csv(file_path_13, index_col = 0)
stats_12_df = pd.read_csv(file_path_12, index_col = 0)
stats_11_df = pd.read_csv(file_path_11, index_col = 0)
stats_10_df = pd.read_csv(file_path_10, index_col = 0)

stats_19_df.head(20)

,Team,Pos,Age,G,AB,R,H,2B,3B,HR,...,CS,BB,SO,SH,SF,HBP,AVG,OBP,SLG,OPS
Player,,,,,,,,,,,,,,,,,,,,,
Whit Merrifield,KC,2B,32,162,681,105,206,41,10,16,...,10,45,126,0,4,5,0.302,0.348,0.463,0.811
Marcus Semien,OAK,SS,30,162,657,123,187,43,7,33,...,8,87,102,0,1,2,0.285,0.369,0.522,0.891
Rafael Devers,BOS,3B,24,156,647,129,201,54,4,32,...,8,48,119,1,2,4,0.311,0.361,0.555,0.916
Jonathan Villar,BAL,2B,30,162,642,111,176,33,5,24,...,9,61,176,2,4,4,0.274,0.339,0.453,0.792
Ozzie Albies,ATL,2B,24,160,640,102,189,43,8,24,...,4,54,112,0,4,4,0.295,0.352,0.500,0.852
Eduardo Escobar,ARI,3B,32,158,636,94,171,29,10,35,...,1,50,130,0,10,3,0.269,0.320,0.511,0.831
Starlin Castro,MIA,2B,31,162,636,68,172,31,4,22,...,2,28,111,0,9,3,0.270,0.300,0.436,0.736
Jose Abreu,CWS,1B,34,159,634,85,180,38,1,33,...,2,36,152,0,10,13,0.284,0.330,0.503,0.833
Jorge Polanco,MIN,SS,28,153,631,107,186,40,7,22,...,3,60,116,2,7,4,0.295,0.356,0.485,0.841


In [3]:
# Creating X & y variables
X = stats_21_df.iloc[:, 3:-2]
y = stats_21_df["OPS"].values

In [4]:
# Scale the data
scaler = StandardScaler().fit(X)
X = scaler.transform(X)

In [5]:
# Creating training, validation, and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

X_train.shape

(478, 17)

In [6]:
#X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
#X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

In [7]:
# Model set-up
number_input_features = 17
hidden_nodes_layer1 = 34
hidden_nodes_layer2 = 5

In [8]:
# Define the LSTM RNN model
model = Sequential()

# Layer 1
model.add(
    Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Layer 2
model.add(Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
model.add(Dense(1, activation="sigmoid"))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [9]:
# Compile the model
model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=[
        "accuracy",
        tf.keras.metrics.TruePositives(name="tp"),
        tf.keras.metrics.TrueNegatives(name="tn"),
        tf.keras.metrics.FalsePositives(name="fp"),
        tf.keras.metrics.FalseNegatives(name="fn"),
        tf.keras.metrics.Precision(name="precision"),
        tf.keras.metrics.Recall(name="recall"),
        tf.keras.metrics.AUC(name="auc"),
    ],
)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [10]:
# Training the model
batch_size = 1000
epochs = 50
training_history = model.fit(
    X_train,
    y_train,
    #validation_data=(X_val, y_val),
    epochs=epochs,
    batch_size=batch_size,
    verbose=1,
)

Epoch 1/50
478/478 [==============================] - 0s 276us/sample - loss: 0.7901 - acc: 0.0209 - tp: 34.0000 - tn: 10.0000 - fp: 0.0000e+00 - fn: 434.0000 - precision: 1.0000 - recall: 0.0726 - auc: 0.1783
Epoch 2/50
478/478 [==============================] - 0s 2us/sample - loss: 0.7828 - acc: 0.0209 - tp: 77.0000 - tn: 10.0000 - fp: 0.0000e+00 - fn: 391.0000 - precision: 1.0000 - recall: 0.1645 - auc: 0.1936
Epoch 3/50
478/478 [==============================] - 0s 4us/sample - loss: 0.7758 - acc: 0.0209 - tp: 89.0000 - tn: 10.0000 - fp: 0.0000e+00 - fn: 379.0000 - precision: 1.0000 - recall: 0.1902 - auc: 0.1959
Epoch 4/50
478/478 [==============================] - 0s 4us/sample - loss: 0.7691 - acc: 0.0188 - tp: 95.0000 - tn: 9.0000 - fp: 1.0000 - fn: 373.0000 - precision: 0.9896 - recall: 0.2030 - auc: 0.1814
Epoch 5/50
478/478 [==============================] - 0s 4us/sample - loss: 0.7626 - acc: 0.0167 - tp: 99.0000 - tn: 8.0000 - fp: 2.0000 - fn: 369.0000 - precision: 0.9802

In [13]:
# Make predictions using the testing data X_test
predicted = model.predict(X_test)

# Evaluate the model
model.evaluate(X_test, y_test)

160/160 [==============================] - 0s 380us/sample - loss: 0.6177 - acc: 0.0312 - tp: 102.0000 - tn: 5.0000 - fp: 0.0000e+00 - fn: 53.0000 - precision: 1.0000 - recall: 0.6581 - auc: 1.0000


[0.6176957249641418, 0.03125, 102.0, 5.0, 0.0, 53.0, 1.0, 0.65806454, 1.0]

In [24]:
real_OPS = stats_19_df["OPS"]

In [20]:
moneyball = pd.DataFrame({
    "Real": real_OPS.ravel(),
    "Predicted": predicted.ravel()
    }, index = stats_19_df.index[-len(real_OPS): ])
moneyball.head()

ValueError: Shape of passed values is (160, 2), indices imply (694, 2)